#Modules Importation and preparation

In [ ]:
#specify the working directory and cd into it
WORK_DIR = ""
% cd ""

In [ ]:
import Clusterer, Global_feature_exractors, Local_features_extractor, Norms, Image, PCA_reduction, Distances
from Dataset_loader import load_dataset_all, load_dataset_IAM
import json, os
import numpy as np
from testing import get_test_sample, accuracy_optimised

In [ ]:
training_object = "training_16Sep"

In [ ]:
with open("./config_train.json") as config_file:
    configuration = json.load(config_file)["Sift_Bow"][training_object]

In [ ]:
BASE_FOLDER = configuration["base_folder"]
if not os.path.exists(BASE_FOLDER):
  os.mkdir(BASE_FOLDER)

#Train Model

In [ ]:
def new_cluster_centers(images, path_to_save, nb_clusters, max_no_improvement):
	hellinger_normalization = Norms.Norm.hellinger_normalization
	local_features_extractor = Local_features_extractor.Local_feature_exractor(hellinger_normalization)

	images_pre = [Image.Image(path_image, local_features_extractor, verbose=False) for path_image in images]
	print("All images prepared!")
 
	Clusterer.Clusterer.fit_new_trainig(images_pre, 
                                        path_to_save, 
                                        nb_clusters=nb_clusters, 
                                        max_no_improvement=max_no_improvement,
                                        verbose=0)

In [ ]:
def choose_cluster_centers(images, max_no_improvement):
  hellinger_normalization = Norms.Norm.hellinger_normalization
  local_features_extractor = Local_features_extractor.Local_feature_exractor(hellinger_normalization)

  images_pre = [Image.Image(path_image, local_features_extractor, verbose=False) for path_image in images]
  print("All images prepared!")

  Clusterer.Clusterer.elbow_method_kmeans(images_pre, max_no_improvement=max_no_improvement)

In [ ]:
_, images = load_dataset_all(WORK_DIR)
nb_images_treat = configuration["nb_images_treat"]

In [ ]:
choose_cluster_centers(images=images[:nb_images_treat], max_no_improvement=configuration["max_no_improvement"])

In [ ]:
nb_clusters_kmeans = 50

In [ ]:
nb_images_treat = configuration["nb_images_treat"]
new_cluster_centers(images=images[:nb_images_treat],
                    path_to_save=BASE_FOLDER+configuration["path_cluster_centers"], 
                    nb_clusters=nb_clusters_kmeans,
                    max_no_improvement=configuration["max_no_improvement"])

#Test Model

In [ ]:
import importlib, Dataset_loader
importlib.reload(Dataset_loader)

In [ ]:
def test_sift_descr_bow(images, writers, cluster_centers_path, accuracy_calculator):
  
  hellinger_normalization = Norms.Norm.hellinger_normalization
  local_features_extractor = Local_features_extractor.Local_feature_exractor(hellinger_normalization)

  clusters_centers = Clusterer.Clusterer.fit_ancient_data(cluster_centers_path)
  bow = Global_feature_exractors.BOW(clusters_centers)

  images_pre = [Image.Image(path_image, local_features_extractor, global_feature_extractor=bow) for path_image in images]

  chi2_distance = Distances.Distance.chi2_distance
  accuracy_value = accuracy_calculator(images_pre, writers, bow, chi2_distance)

  return accuracy_value

In [ ]:
writers, images = load_dataset_all(WORK_DIR)
X_test, y_test = get_test_sample(images, writers, configuration["test_sample_size"])

In [ ]:
accuracy_value= test_sift_descr_bow(images=X_test, 
                                    writers=y_test, 
                                    cluster_centers_path=BASE_FOLDER+configuration["path_cluster_centers"],
                                    accuracy_calculator=accuracy_optimised)

print()
print("Accuracy value for", training_object," :",accuracy_value*100,"%")